<br>

# Unidades e Imóveis

O TJSP, em sua API e interface do usuário, utiliza com sinônimos os conceitos de "Unidades" e "Imóveis".

<br>

Michel Metran\
Data: 18.06.2024\
Atualizado em: 18.06.2024


In [ ]:
#!pip3 install requests-ip-rotator
#!pip3 install traquitanas

In [ ]:
import concurrent
import os
import re
from typing import Literal

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from open_geodata import geo
from requests_ip_rotator import ApiGateway

from sp_tjsp_divadmin.my_functions import (
    adjust_columns,
    find_text_between_parenthesis,
    keep_numbers,
    strip_accents,
)
from sp_tjsp_divadmin.my_paths import output_path_tab

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
# from selenium.webdriver.firefox.service import Service as FirefoxService
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.wait import WebDriverWait
# from traquitanas.scrapping import adds, gecko

# from sp_tjsp_divadmin.my_driver import Driver

# from sp_tjsp_divadmin.my_paths import (
#     adds_path,
#     driver_path,
#     logs_path,
# )

In [ ]:
load_dotenv()

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

<br>

---

## Imóveis

Na primeira tentativa, montamos uma lista de termos que iremos utiliza para a pesquisa. Juntamos tanto a lista de Unidades, obtida em consulta anterior, como a lista com o nome de municípios.


In [ ]:
# Preciso listar Imóveis para obter o código
df_unidades = pd.read_csv(
    output_path_tab / 'Unidades, Municípios e Comarcas.csv'
)
list_unidades = list(df_unidades['unidades'])

# Passa para lowercase
list_unidades = [x.lower() for x in list_unidades]

# Pega cada palavra
list_unidades_words = [
    word for phrase in list_unidades for word in phrase.split()
]

# Retira duplicados
list_unidades_words = list(set(list_unidades_words))

# Pega maior que 4 caracters
list_unidades_words = [x for x in list_unidades_words if len(x) >= 4]

# Results
print(len(list_unidades_words))
list_unidades_words[:10]

In [ ]:
# Cria Lista
df_geo_mun = geo.load_dataset('tab.sp.tab_municipio_nome')
lista_municipios = list(df_geo_mun['municipio_nome'])

# Passa para lowercase
lista_municipios = [x.lower() for x in lista_municipios]

# Pega cada palavra
lista_municipios_words = [
    word for phrase in lista_municipios for word in phrase.split()
]

# Retira duplicados
lista_municipios_words = list(set(lista_municipios_words))

# Pega maior que 4 caracters
lista_municipios_words = [x for x in lista_municipios_words if len(x) >= 4]

# Results
print(len(lista_municipios_words))
lista_municipios_words[:10]

In [ ]:
list_search = list_unidades_words + lista_municipios_words
list_search = list(set(list_search))

print(len(list_search))
list_search[0:10]

Com a lista pronta, instânciamos uma API na AWS, para que as requisições não sejam bloqueadas.


In [ ]:
# Cria Gateway
gateway = ApiGateway(
    site='https://www.tjsp.jus.br',
    access_key_id=aws_access_key_id,
    access_key_secret=aws_secret_access_key,
    regions=['sa-east-1'],
    verbose=True,
)
gateway.pool_connections = 10
gateway.pool_maxsize = 10
gateway.start()

# Cria Session
session = requests.Session()
session.mount(prefix='https://www.tjsp.jus.br', adapter=gateway)

In [ ]:
def get_id_unidade(
    unidade,
    req_type: Literal['basic', 'aws'] = 'aws',
    req_session=session,
) -> pd.DataFrame:
    """
    Obtem o código (ou ID) das Unidades (ou Imóveis) do TJSP

    :param unidade: nome da Unidade, ou trecho do nome
    :return: tabela com os valores
    """
    # Get Data
    if req_type == 'basic':
        r = requests.post(
            'https://www.tjsp.jus.br/AutoComplete/ListarImoveis',
            json={'texto': unidade},
        )
    elif req_type == 'aws':
        r = req_session.post(
            'https://www.tjsp.jus.br/AutoComplete/ListarImoveis',
            json={'texto': unidade},
        )

    # Se é lista vazia, ignora
    if r.json() == 'listaVazia':
        print(f'----------- Vazio em {unidade}')

    # Se não é, registra!
    else:
        try:
            df = pd.DataFrame(r.json())
            df = df.rename(
                mapper={
                    'Codigo': 'id_unidade',
                    'Descricao': 'unidade',
                },
                axis='columns',
            )
            return df
        except:
            print(f'>>>>>>>> Errou em {unidade}')

In [ ]:
get_id_unidade(unidade='Santos', req_type='basic')

Inicialmente eu tentei utilizar um método básico para consultas, utilizando a biblioteca `requests`.

```
list_dfs = []
for i in list_unidades:
    list_dfs.append(get_id_unidade(unidade=i))


df_unidades_ids = pd.concat(objs=list_dfs, ignore_index=True)
df_unidades_ids
```

<br>

Contudo, não estava nada eficente. Rodou trinta minutos e não baixou tudo... dai cancelei e procurei alternativas.


In [ ]:
# Encerra o worker
# gateway.shutdown()

In [ ]:
get_id_unidade(unidade='Santos', req_type='aws')

In [ ]:
# Parameters
MAX_THREADS = 4
list_dfs = []
list_futures = []

# Paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Cria Lista de Tarefas
    for un in list_search:
        futures = executor.submit(get_id_unidade, un)
        list_futures.append(futures)

    # Executa Lista de Tarefas
    for future in concurrent.futures.as_completed(list_futures):
        print(future.result())
        list_dfs.append(future.result())

# Encerra o worker
#gateway.shutdown()

Com isso descobrimos que apesar da "força bruta" não foi possível capturar todas as Unidades.


In [ ]:
# Junta
df_unidades_ids = pd.concat(objs=list_dfs, ignore_index=True)
df_unidades_ids = df_unidades_ids.drop_duplicates()
df_unidades_ids = df_unidades_ids.sort_values(by='id_unidade', ascending=True)
df_unidades_ids = df_unidades_ids.reset_index(drop=True)

# Resultados
df_unidades_ids.info()
display(df_unidades_ids.head())
display(df_unidades_ids.tail())

In [ ]:
# Pesquisar uma Unidade Específica
df_unidades_ids[df_unidades_ids['id_unidade'] == 259]

In [ ]:
# filename = 'Unidades Id'

# # Salva
# df_unidades_ids.to_csv(output_path_tab / f'{filename}.csv', index=False)
# df_unidades_ids.to_excel(
#     output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
# )

<br>

---

## Detalhes dos Imóveis / Unidades

A pesquisa acima, que não mostrou bom resultado, indicou que o número de imóveis é de aproximadamente 1000! Isso é um ótimo indicativo!

Construimos funções que farão a requisição dos dados.


In [ ]:
def get_detalhes_unidades(
    id_unidade,
    req_type: Literal['basic', 'aws'] = 'aws',
    req_session=session,
):
    """
    _summary_
    # Uma vez com o Código do Imóvel, consigo obter detalhes
    parmsEntrada=827&codigoTipoBusca=2
    Aqui tem tudo que eu preciso!!!!
    :param id_unidade: _description_
    :type id_unidade: _type_
    :param req_type: _description_, defaults to 'session'
    :type req_type: Literal[&#39;basic&#39;, &#39;session&#39;], optional
    :param req_session: _description_, defaults to session
    :type req_session: _type_, optional
    :return: _description_
    :rtype: _type_
    """

    if req_type == 'basic':
        r = requests.post(
            'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
            json={'parmsEntrada': id_unidade, 'codigoTipoBusca': 2},
        )

    elif req_type == 'aws':
        r = req_session.post(
            'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
            json={'parmsEntrada': id_unidade, 'codigoTipoBusca': 2},
        )

    soup = BeautifulSoup(r.text, 'html.parser')
    return {'id_unidade': id_unidade, 'soup': soup}

In [ ]:
def get_detalhes_unidades_soup(res_get_detalhes_unidades):
    """
    _summary_

    :param soup: _description_
    :return: _description_
    """
    # Trbalha com input
    soup = res_get_detalhes_unidades['soup']
    id_unidade = res_get_detalhes_unidades['id_unidade']

    # Parâmetros
    try:
        unidade = soup.find(name='h3', attrs={'id': 'imovelNome'}).text.strip()
    except:
        unidade = 'Sem nome'

    endereco = (
        soup.find('dt', string=re.compile('.*Endereço.*', flags=re.DOTALL))
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    telefone = (
        soup.find('dt', string=re.compile('.*Telefone.*', flags=re.DOTALL))
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    fax = (
        soup.find('dt', string=re.compile('.*Fax.*', flags=re.DOTALL))
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    email: str = (
        soup.find('dt', string=re.compile('.*E-mail.*', flags=re.DOTALL))
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    cj: str = (
        soup.find(
            'dt',
            string=re.compile('.*Circunscrição Judiciária.*', flags=re.DOTALL),
        )
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    num_varas_instaladas = (
        soup.find(
            'dt',
            string=re.compile(
                '.*Número de Varas Instaladas.*', flags=re.DOTALL
            ),
        )
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    entrancia = (
        soup.find('dt', string=re.compile('.*Entrância.*', flags=re.DOTALL))
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    # re.compile('.*Comarca.*', flags=re.DOTALL)
    comarca = (
        soup.find('dt', string=re.compile('.*Comarca.*', flags=re.DOTALL))
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    dist_capital = (
        soup.find(
            'dt', string=re.compile('.*Distância da Capital.*', flags=re.DOTALL)
        )
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    tensao_eletrica = (
        soup.find(
            'dt', string=re.compile('.*Tensão Elétrica.*', flags=re.DOTALL)
        )
        .parent.find(name='dd')
        .find(name='span')
        .text.strip()
    )

    tj_dict = {
        'id_unidade': id_unidade,
        'unidade': unidade,
        'endereco': endereco,
        'telefone': telefone,
        'fax': fax,
        'email': email,
        'cj': cj,
        'num_varas_instaladas': num_varas_instaladas,
        'entrancia': entrancia,
        'comarca': comarca,
        'dist_capital': dist_capital,
        'tensao_eletrica': tensao_eletrica,
    }

    return tj_dict

E, obtendo os dados sem o serviço da AWS temos o seguinte resultado!


In [ ]:
id_unidade = 88
results = get_detalhes_unidades(id_unidade=id_unidade, req_type='basic')

# Trata Resultado
soup = results['soup']
print(soup.prettify())

# Detalhes
get_detalhes_unidades_soup(res_get_detalhes_unidades=results)

In [ ]:
# comarca = (
#     soup.find('dt', string=re.compile('.*Comarca.*', flags=re.DOTALL))
#     .parent.find(name='dd')
#     .find(name='span')
#     .text.strip()
# )
# comarca

In [ ]:
def get_unidades(id_unidade=id_unidade):

    results = get_detalhes_unidades(
        id_unidade=id_unidade, req_type='aws', req_session=session
    )

    # Trata Resultado
    # soup = results['soup']
    # print(soup.prettify())

    # Resultado
    dd = get_detalhes_unidades_soup(res_get_detalhes_unidades=results)
    return dd

In [ ]:
id_unidade = 88
get_unidades(id_unidade=id_unidade)

In [ ]:
# Parameters
MAX_THREADS = 4
list_dfs = []
list_futures = []

# Paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Cria Lista de Tarefas
    for i in range(1, 1500):
        futures = executor.submit(get_unidades, i)
        list_futures.append(futures)

    # Executa Lista de Tarefas
    for future in concurrent.futures.as_completed(list_futures):
        print(future.result())
        list_dfs.append(future.result())

# Encerra o worker
gateway.shutdown()

Fazemos algumas correções no _dataframe_.


In [ ]:
# Monta Tabela
df_unidades_infos = pd.DataFrame(list_dfs)
df_unidades_infos = df_unidades_infos.drop_duplicates()
df_unidades_infos = df_unidades_infos.sort_values(
    by='id_unidade', ascending=True
)
df_unidades_infos = df_unidades_infos.reset_index(drop=True)

# Results
df_unidades_infos.info()
display(df_unidades_infos.head())
display(df_unidades_infos.tail())

Ajustamos telefone e fax


In [ ]:
# Remove ; do fim da coluna
df_unidades_infos['telefone'] = df_unidades_infos['telefone'].str.replace(
    r'\;$', '', regex=True
)
df_unidades_infos['fax'] = df_unidades_infos['fax'].str.replace(
    r'\;$', '', regex=True
)

Splitamos o endereço

Consegui splitar no "CEP", em duas partes.\
A segunda parte eu "re-splitei" em CEP, Municípioe e UF.

Não há como "re-splitei" a primeira parte, pois no campo "Lougradouro" dá conflito!


In [ ]:
# df_unidades_infos [['endereco_lougradouro', 'endereco_bairro', 'endereco_cep', 'endereco_uf']]=
df_unidades_infos[['endereco_lougradouro', 'endereco_p2']] = df_unidades_infos[
    'endereco'
].str.split(' - CEP ', n=1, expand=True)
df_unidades_infos[['endereco_cep', 'endereco_municipio', 'endereco_uf']] = (
    df_unidades_infos['endereco_p2'].str.split(' - ', n=3, expand=True)
)

# Remove Columns
df_unidades_infos = df_unidades_infos.drop(
    labels=['endereco_p1', 'endereco_p2', 'endereco'],
    inplace=False,
    errors='ignore',
    axis='columns',
)

In [ ]:
# df_unidades_infos[df_unidades_infos['endereco'].str.contains(' - CEP')]

# Lougradouro / Bairro
# df = df_unidades_infos['endereco_lougradouro'].str.split(' - ', n=3, expand=True)
# df[~df[3].isna()]
# df

In [ ]:
# aa = 'Avenida São José , 431 - Centro - CEP 17450-001 - Gália - SP'
# aa.split(' - ')

Strip all columns


In [ ]:
# Aplica strip em todo o dataframe
df_unidades_infos = df_unidades_infos.map(
    lambda x: x.strip() if isinstance(x, str) else x
)

In [ ]:
mask = (
    (df_unidades_infos['unidade'] == '')
    & (df_unidades_infos['telefone'] == 'Não Informado')
    & (df_unidades_infos['fax'] == 'Não Informado')
    & (df_unidades_infos['email'] == 'Não Informado')
    & (df_unidades_infos['cj'] == 'Não Informado')
    & (df_unidades_infos['entrancia'] == 'Não Informado')
    & (df_unidades_infos['comarca'] == 'Não Informado')
    & (df_unidades_infos['dist_capital'] == 'Não Informado')
    & (df_unidades_infos['tensao_eletrica'] == 'Não Informado')
)
df_unidades_infos = df_unidades_infos[~mask]
df_unidades_infos.head()

Varas

In [ ]:
df_unidades_infos['num_varas_instaladas'] = df_unidades_infos['num_varas_instaladas'].astype(int)

Dá pra juntar Comarcas?


In [ ]:
len(set(df_unidades_infos['comarca']))

<br>

### Comarcas

In [ ]:
# Read Data
df_tjsp_com = pd.read_csv(output_path_tab / 'Comarcas.csv')

# Drop Entrância
df_tjsp_com = df_tjsp_com.drop(labels='entrancia', axis='columns')

# Ajustes
df_tjsp_com = adjust_columns(df=df_tjsp_com, column_ajust='comarca_tjsp')

# Results
df_tjsp_com.info()
df_tjsp_com.head()

In [ ]:
# Ajusta
df_unidades_infos = adjust_columns(df=df_unidades_infos, column_ajust='comarca')

In [ ]:
df_merge = pd.merge(
    left=df_unidades_infos,
    right=df_tjsp_com,
    left_on='comarca_temp',
    right_on='comarca_tjsp_temp',
    how='left',
)

# Deleta
df_merge = df_merge.drop(
    labels=['comarca', 'comarca_temp', 'comarca_tjsp', 'comarca_tjsp_temp'],
    axis='columns',
    errors='ignore',
)

# Renomeia
df_merge.info()
df_merge.head()

Salvar


In [ ]:
filename = 'Unidades'

# Salva
df_merge.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_merge.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)

<br>

---

## Entrâncias


In [ ]:
df_unidades = pd.read_csv(output_path_tab / 'Unidades.csv')
df_unidades.head(2)

In [ ]:
df_entrancias = df_unidades[['id_comarca', 'entrancia']]

# Deleta Duplicatos
df_entrancias = df_entrancias.drop_duplicates()

# Remove Null
df_entrancias = df_entrancias[~df_entrancias['id_comarca'].isnull()]

# Convert para Int
df_entrancias['id_comarca'] = df_entrancias['id_comarca'].astype(int)

# Resultados
df_entrancias.info()
df_entrancias.head()

Adiciona dados de entrância as comarcas

In [ ]:
df_entrancias[df_entrancias['id_comarca'] == 3501608]

In [ ]:
# Read Data
df_tjsp_com = pd.read_csv(output_path_tab / 'Comarcas.csv')

# Drop Entrância
df_tjsp_com = df_tjsp_com.drop(labels='entrancia', axis='columns')

# Results
df_tjsp_com.info()
df_tjsp_com.head()

df_tjsp_com[df_tjsp_com['id_comarca'] == 3501608]

In [ ]:
df_merge = pd.merge(
    left=df_tjsp_com,
    right=df_entrancias,
    left_on='id_comarca',
    right_on='id_comarca',
    how='inner',
)

df_merge

Foi

In [ ]:
filename = 'Comarcas'

# Salva
df_merge.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_merge.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)